# Contorl of PRR (UnderActued setting): applicability of Input Decouplig through Collocated Form

###    Abstract 
Project for the class of Underactuated robotics of professor L. Lanari.

Regulation task for a planar robot through Cartesian force applied at the end-effector level. \\
After prove that the definition of Collocated Form suit to this kind of structure, we will perform the regulation task and obtain a partition of the actuation matrix A(q) like :
$$ A(q) = 
\begin{bmatrix}
     I &\\
     0 &
\end{bmatrix}
$$


The kind of control that can be achive with this procedure could be easily apply to medical tasks, soft robots and actuator fault tolerant system in cooperative tasks. 

### On the Integrability condition for the collacated form

Defining the passive output as:
$$ \dot y = A^{T}(q)\dot q $$
end considering the dynamic model of such robot as
$$
\begin{equation}
 M(q)\ddot q + c(q, \dot q) = J^T F
\end{equation}
$$
we can easily reconize that the $A^T$ matrix is nothing else that the Jacobian of the system, and so we can conclude that out passive output $\dot y$ is the velocity of the end effector.


Thanks to this result, we can claim that any system with similar dynamics, can be considered Collocated, AKA admit a solution to the Input Decoupling problem through a change of cordinate

In [1]:
"""## importing"""
import numpy as np

from pydrake.common.value import Value
from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.systems.framework import DiagramBuilder, LeafSystem, BasicVector
from pydrake.multibody.math import SpatialForce
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, ExternallyAppliedSpatialForce, ExternallyAppliedSpatialForceMultiplexer
from pydrake.multibody.parsing import Parser
from pydrake.all import Simulator


# pydot = interface to graphViz,consist in a parser for the DotFormat used by GraphViz
from pydot import graph_from_dot_data

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
class Controller(LeafSystem):

    def __init__(self): 
        LeafSystem.__init__(self)
        self.DeclareVectorOutputPort("y", BasicVector(2), self.CalcOutputY)
        self.t = 50
    def CalcOutputY(self, context, output): 
        q1dotdot = 0
        q2dotdot = 0
        output.SetFromVector([q1dotdot, q2dotdot])
    

class CartesianForceEE_Sys(LeafSystem):
    def __init__(self, end_effector_id):
        LeafSystem.__init__(self)
        ## input
        self.DeclareVectorInputPort("position_req", BasicVector(2))

        ##output
        self.force_obj = ExternallyAppliedSpatialForce()
        self.force_obj.body_index = end_effector_id
        force_cls = Value[list[ExternallyAppliedSpatialForce]]
        self.DeclareAbstractOutputPort("F-cartesian", 
                                       lambda: force_cls(), 
                                       self.CalcSpatialForce)

    def CalcSpatialForce(self, context, applied_force):
        F = SpatialForce(tau=np.zeros(3), f=np.array([1., 0., 0.]))
        self.force_obj.F_Bq_W = F
        applied_force.set_value([self.force_obj])


# building diagram, connecting port


In [24]:
builder = DiagramBuilder()

In [25]:

time_step = 0.003
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
Parser(plant, scene_graph).AddModels("../sdf_model/RR_planar.sdf")
plant.Finalize()


In [26]:
## controller
end_effector_id = plant.GetBodyByName('end-effector').index()

CartesianController_sys = CartesianForceEE_Sys(end_effector_id)
builder.AddSystem(CartesianController_sys)
builder.Connect(CartesianController_sys.get_output_port(0), plant.get_applied_spatial_force_input_port())

In [27]:
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph.get_query_output_port(),
    meshcat
)

In [28]:
diagram = builder.Build()
diagram_root_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_root_context)
Controller_context = CartesianController_sys.GetMyContextFromRoot(diagram_root_context)    

In [30]:
## set initial position
plant.GetJointByName('q1').set_angle(plant_context, -np.pi)
plant.GetJointByName('q2').set_angle(plant_context, np.pi/2)
diagram.ForcedPublish(diagram_root_context)

## simulate actuation fault
plant.get_actuation_input_port().FixValue(plant_context, np.zeros(2))


# Simulate and save data

In [31]:
simulator = Simulator(diagram, diagram_root_context)
simulator.AdvanceTo(50)

In [10]:
def save_plant_svg_graph(diagram, destination_path):
    dot_format_graph = graph_from_dot_data(diagram.GetGraphvizString())[0]
    binary_data_svg = dot_format_graph.create_svg()
    with open(destination_path, 'wb') as f:
        f.write(binary_data_svg)

save_plant_svg_graph(diagram, destination_path="../log/RR_planar.svg")